![title](img\Afeka2.png)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
haaretz_work_dir = 'HAARETZ_TEST'
import os
for dirname, _, filenames in os.walk(haaretz_work_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


HAARETZ_TEST\apr2-13-90\1.txt
HAARETZ_TEST\apr2-13-90\10.txt
HAARETZ_TEST\apr2-13-90\2.txt
HAARETZ_TEST\apr2-13-90\3.txt
HAARETZ_TEST\apr2-13-90\4.txt
HAARETZ_TEST\apr2-13-90\5.txt
HAARETZ_TEST\apr2-13-90\6.txt
HAARETZ_TEST\apr2-13-90\7.txt
HAARETZ_TEST\apr2-13-90\8.txt
HAARETZ_TEST\apr2-13-90\9.txt


In [4]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
import matplotlib                  # 2D Plotting Library        
plt.style.use('fivethirtyeight')
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD

stop_words=set(nltk.corpus.stopwords.words('english'))

#hebrew stop-words
import codecs
he_stop_words = []
with codecs.open('heb_stopwords.txt', 'r', encoding='utf-8', errors='ignore') as file:
            he_stop_words = file.read().splitlines()

In [5]:
import codecs

df = pd.DataFrame(columns=['Text', 'FN'])
i = 0
for dirname, _, filenames in os.walk(haaretz_work_dir):
    for filename in filenames:
        data = ''
        file_name = str(os.path.join(dirname, filename))
        with codecs.open(file_name, 'r', encoding='utf-8', errors='ignore') as file:
            data = file.read().replace('\n', ' ')
        df = df.append({'Text': data, 'FN': file_name}, ignore_index=True)

In [6]:
df.head()

,Text,FN
0,את העונה הנוכחית החלה הפועל בת ים כאלופה ומחזי...,HAARETZ_TEST\apr2-13-90\1.txt
1,בערב שבו נערכו משחקי גמר גביע המדינה בכדורעף מ...,HAARETZ_TEST\apr2-13-90\10.txt
2,כל הצלחה ספורטיבית בנויה על גידול וטיפוח ילדים...,HAARETZ_TEST\apr2-13-90\2.txt
3,בתוך ים השחקנים המצטיינים והבינלאומיים של הפוע...,HAARETZ_TEST\apr2-13-90\3.txt
4,"אף אחד בבת ים לא יגיד את זה בקול רם , אבל איש ...",HAARETZ_TEST\apr2-13-90\4.txt


In [7]:
index = df.index
number_of_rows = len(index)
print(number_of_rows)

10


In [8]:
new1 = df[['Text']].iloc[:27140].copy()

In [9]:
import collections
texts = df['Text'].str.lower()
word_counts = collections.Counter(word_tokenize('\n'.join(texts)))


In [10]:
print(len(word_counts))
word_counts.most_common()

2189


[(',', 379),
 ('.', 275),
 ('של', 82),
 ('את', 69),
 ('ים', 62),
 ('``', 57),
 ('בת', 49),
 ('לא', 46),
 ('על', 40),
 (':', 36),
 ('הפועל', 28),
 ('גם', 26),
 ('העונה', 25),
 ('הכדורעף', 25),
 ('המדינה', 22),
 ('עם', 20),
 ("''", 18),
 ('הקבוצה', 18),
 ('הרשקו', 17),
 ('-', 17),
 ('כל', 15),
 ('(', 14),
 (')', 14),
 ('הוא', 14),
 ('קבוצת', 14),
 ('האבטלה', 14),
 ('רק', 13),
 ('זה', 13),
 ('אבל', 13),
 ('יש', 13),
 ('במסגרת', 12),
 ('בליגה', 12),
 ('גביע', 11),
 ('וגם', 11),
 ('יותר', 11),
 ('היה', 11),
 ('הגביע', 11),
 ('משחק', 11),
 ('עד', 11),
 ('בבת', 11),
 ('בגביע', 11),
 ('הלאומית', 11),
 ('היו', 11),
 ('כבר', 10),
 ('3', 10),
 ('שחקני', 10),
 ('שחקנים', 10),
 ('האגודה', 10),
 ('כך', 10),
 ('ענף', 10),
 ('פביאן', 10),
 ('לאחר', 10),
 ('השחקנים', 9),
 ('היתה', 9),
 ('גמר', 9),
 ('אשר', 9),
 ('במהלך', 9),
 ('ב', 9),
 ('בית', 9),
 ('היא', 9),
 ('אחד', 8),
 ('הבת', 8),
 ('סמו', 8),
 ('א', 8),
 ('כי', 8),
 ('משחקי', 8),
 ('זו', 8),
 ('מטה', 8),
 ('האליפות', 8),
 ('כדורעף', 8),
 ('הנבחר

### DATA CLEANING & PRE-PROCESSING
Data pre-processing. We use stemming to remove special characters. 
Also the stop words have been removed along with the words with length shorter than 3 characters to reduce some stray words.

In [11]:
def clean_text(text):
    lancaster=LancasterStemmer()
    word_tokens=word_tokenize(text)
    tokens=[lancaster.stem(w) for w in word_tokens if w not in he_stop_words and len(w)>3 and not w.isnumeric()]
    cleaned_text=" ".join(tokens)
    return cleaned_text

In [12]:
#it takes time
new1['cleaned_text']=new1['Text'].apply(clean_text)

In [13]:
import collections
texts = new1['cleaned_text'].str.lower()
cleaned_text_word_counts = collections.Counter(word_tokenize('\n'.join(texts)))
#307548
print(len(cleaned_text_word_counts))
cleaned_text_word_counts.most_common()


1737


[('הפועל', 28),
 ('העונה', 25),
 ('הכדורעף', 25),
 ('המדינה', 22),
 ('הקבוצה', 18),
 ('הרשקו', 17),
 ('קבוצת', 14),
 ('האבטלה', 14),
 ('במסגרת', 12),
 ('בליגה', 12),
 ('גביע', 11),
 ('הגביע', 11),
 ('בגביע', 11),
 ('הלאומית', 11),
 ('שחקני', 10),
 ('שחקנים', 10),
 ('האגודה', 10),
 ('פביאן', 10),
 ('השחקנים', 9),
 ('במהלך', 9),
 ('משחקי', 8),
 ('האליפות', 8),
 ('כדורעף', 8),
 ('הנבחרת', 8),
 ('קבוצות', 8),
 ('הספר', 8),
 ('העבודה', 8),
 ('ימים', 7),
 ('הליגה', 7),
 ('הפלייאוף', 7),
 ('זכתה', 7),
 ('המאמן', 7),
 ('יעקב', 7),
 ('הגמר', 7),
 ('במשחק', 7),
 ('מזכיר', 7),
 ('באגודה', 7),
 ('הנשים', 7),
 ('לכדורעף', 7),
 ('בנות', 7),
 ('אקרמן', 6),
 ('בקבוצה', 6),
 ('קרית', 6),
 ('אירופה', 6),
 ('המועדון', 6),
 ('הבנות', 6),
 ('שירות', 6),
 ('להפועל', 5),
 ('יוסי', 5),
 ('הנוער', 5),
 ('שנות', 5),
 ('בגמר', 5),
 ('אלון', 5),
 ('רוני', 5),
 ('מזעקי', 5),
 ('בהפועל', 5),
 ('בענף', 5),
 ('יכולת', 5),
 ('במספר', 5),
 ('מיכאל', 5),
 ('הכדורגל', 5),
 ('אולם', 5),
 ('לליגה', 5),
 ('ולדינגר', 5),
 ('

In [14]:
new1.head()

,Text,cleaned_text
0,את העונה הנוכחית החלה הפועל בת ים כאלופה ומחזי...,העונה הנוכחית החלה הפועל כאלופה ומחזיקת גביע ת...
1,בערב שבו נערכו משחקי גמר גביע המדינה בכדורעף מ...,בערב נערכו משחקי גביע המדינה בכדורעף מיהר בנצי...
2,כל הצלחה ספורטיבית בנויה על גידול וטיפוח ילדים...,הצלחה ספורטיבית בנויה גידול וטיפוח ונערים הפטנ...
3,בתוך ים השחקנים המצטיינים והבינלאומיים של הפוע...,השחקנים המצטיינים והבינלאומיים הפועל בכדורעף נ...
4,"אף אחד בבת ים לא יגיד את זה בקול רם , אבל איש ...",יגיד בקול יכחיש העובדה שחקני השישיה הראשונה הפ...


We can clearly see the difference after removal of stopwords and some shorter words.

### EXTRACTING THE FEATURES AND CREATING THE DOCUMENT-TERM-MATRIX ( DTM )
A document-term matrix or term-document matrix is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.

1) LSA is generally implemented with Tfidf values everywhere and not with the Count Vectorizer.

2) max_features depends on your computing power and also on eval. metric (coherence score is a metric for topic model). Try the value that gives best eval. metric and doesn't limits processing power.

3) Default values for min_df & max_df worked well.

4) Can try different values for ngram_range.

In [15]:
vect = TfidfVectorizer(stop_words=stop_words, max_features=1000)

In [16]:
vect_text = vect.fit_transform(new1['cleaned_text'])

In [17]:
print(vect.get_feature_names())

['אבטלה', 'אגודה', 'אובדן', 'אולם', 'אחראי', 'איגוד', 'אינה', 'איפשר', 'אירופה', 'איתור', 'אלבס', 'אלון', 'אליו', 'אליפויות', 'אלפי', 'אמיר', 'אסור', 'אקרמן', 'ארבע', 'ארצית', 'אשמים', 'באגודה', 'באולם', 'באותה', 'באליפות', 'בארצות', 'בגביע', 'בגיל', 'בגמר', 'בהפועל', 'בהתחלה', 'בוגרות', 'בוגרים', 'בזמן', 'בחודש', 'בחודשי', 'בחצי', 'ביכולתו', 'ביציע', 'בישראל', 'בכדורעף', 'בלבד', 'בליגה', 'במדינות', 'במהלך', 'במחלקת', 'במסגרת', 'במספר', 'במערב', 'במשחק', 'במשחקי', 'במשך', 'בנאמנות', 'בנות', 'בנים', 'בנצי', 'בסגל', 'בעונה', 'בעיות', 'בעיר', 'בענף', 'בעתיד', 'בפלייאוף', 'בקבוצה', 'בקבוצת', 'בקרב', 'ברצינות', 'ברציפות', 'בשבוע', 'בשורה', 'בשני', 'בשתי', 'בתולדותיה', 'בתחום', 'בתיכון', 'גבוה', 'גביע', 'גביעי', 'גופני', 'גידול', 'גלוזברג', 'גרינברג', 'דאגה', 'דופן', 'דורשי', 'דרכה', 'האבטלה', 'האגודה', 'האוצר', 'האחרון', 'האחרות', 'האימון', 'האיתור', 'האלופה', 'האליפות', 'הארצית', 'הבאה', 'הבוגרות', 'הבוגרים', 'הבוגרת', 'הבית', 'הבלתי', 'הבנות', 'הבנייה', 'הברית', 'הגביע', 'הגברים', 'הגובה'

We can now see the most frequent and rare words in the cleaned_text column based on idf score. The lesser the value; more common is the word in the column.

In [18]:
print(vect_text.shape)
type(vect_text)


(10, 1000)


scipy.sparse.csr.csr_matrix

In [19]:

idf=vect.idf_

In [20]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
#print(dd['יותר'])
#print(dd['zz'])  # police is most common and forecast is least common among the news headlines.

הפועל תקרית


In [21]:
new1['cleaned_text'].head()

0    העונה הנוכחית החלה הפועל כאלופה ומחזיקת גביע ת...
1    בערב נערכו משחקי גביע המדינה בכדורעף מיהר בנצי...
2    הצלחה ספורטיבית בנויה גידול וטיפוח ונערים הפטנ...
3    השחקנים המצטיינים והבינלאומיים הפועל בכדורעף נ...
4    יגיד בקול יכחיש העובדה שחקני השישיה הראשונה הפ...
Name: cleaned_text, dtype: object

### TOPIC MODELLING

Latent Semantic Analysis (LSA)
The first approach that I have used is the LSA. LSA is basically singular value decomposition.

SVD decomposes the original DTM into three matrices S=U.(sigma).(V.T). Here the matrix U denotes the document-topic matrix while (V) is the topic-term matrix.

Each row of the matrix U(document-term matrix) is the vector representation of the corresponding document. The length of these vectors is the number of desired topics. Vector representation for the terms in our data can be found in the matrix V (term-topic matrix).

So, SVD gives us vectors for every document and term in our data. The length of each vector would be k. We can then use these vectors to find similar words and similar documents using the cosine similarity method.

We can use the truncatedSVD function to implement LSA. The n_components parameter is the number of topics we wish to extract. The model is then fit and transformed on the result given by vectorizer.

Lastly note that LSA and LSI (I for indexing) are the same and the later is just sometimes used in information retrieval contexts.

In [22]:
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [23]:
print(lsa_top[0])
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[ 0.58810788  0.2940621   0.0484848  -0.02908242  0.07309611  0.15740604
 -0.03312529 -0.05900063  0.70895074 -0.16477505]
(10, 10)


In [24]:
l = lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ", i, " : ", topic*100)

Document 0 :
Topic  0  :  58.8107883033335
Topic  1  :  29.406210411567457
Topic  2  :  4.848479504968077
Topic  3  :  -2.9082422400553516
Topic  4  :  7.30961104046193
Topic  5  :  15.740604319313528
Topic  6  :  -3.312529290693261
Topic  7  :  -5.900062570214976
Topic  8  :  70.8950741601999
Topic  9  :  -16.477505127285582


> Similalry for other documents we can do this. However note that values don't add to 1 as in LSA it is not probabiltiy of a topic in a document.

In [25]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 0.00169054  0.03153501  0.03057974 ...  0.0200941   0.00169054
   0.02504175]
 [-0.00149237 -0.04099478  0.01004874 ...  0.01756391 -0.00149237
   0.00303632]
 [ 0.03570867 -0.01105639  0.00253493 ...  0.00325726  0.03570867
   0.00616233]
 ...
 [-0.00014401  0.02835674 -0.00731603 ... -0.00490363 -0.00014401
   0.02332841]
 [-0.002358    0.00694978  0.0328003  ...  0.06302913 -0.002358
  -0.03252769]
 [-0.00035165  0.04629447 -0.03057588 ... -0.01557731 -0.00035165
   0.00180836]]


Now we can get a list of the important words for each of the 10 topics as shown. For simplicity here I have shown 10 words for each topic.

In [26]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:5]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
העונה הכדורעף הפועל שחקני הרשקו 

Topic 1: 
שחקני הנבחרת בלבד המאמן אשמים 

Topic 2: 
האבטלה העבודה שירות בישראל הבנייה 

Topic 3: 
מזעקי מזכיר מבחינה הפועלים בנצי 

Topic 4: 
בכדורעף זומן גלוזברג רוני יוסי 

Topic 5: 
בנות ולדינגר הבנות ברציפות השלישית 

Topic 6: 
אשמים בנות המדינה גביע ולדינגר 

Topic 7: 
האיתור והמיון מלאכת הספר בעיר 

Topic 8: 
ימים הגמר נצחונות המשחקים חילוף 

Topic 9: 
הנבחרת ההכנות העובדה תחילת פביאן 



### Latent Dirichlet Allocation (LDA)
LDA is the most popular technique.The topics then generate words based on their probability distribution. Given a dataset of documents, LDA backtracks and tries to figure out what topics would create those documents in the first place.[](http://)

In [27]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
# n_components is the number of topics

In [28]:
lda_top=lda_model.fit_transform(vect_text)

In [29]:
print(lda_top.shape)  # (no_of_doc,no_of_topics)
print(lda_top[1])

(10, 10)
[0.88557086 0.01271437 0.01271435 0.01271429 0.01271444 0.01271459
 0.01271436 0.01271429 0.01271427 0.01271419]


In [30]:
sum=0
for i in lda_top[0]:
  sum=sum+i
print(sum)

1.0


> Note that the values in a particular row adds to 1. This is beacuse each value denotes the % contribution of the corressponding topic in the document.

In [31]:
# composition of doc 0 for eg
for d in range(1):
    print("Document: ", d)
    for i,topic in enumerate(lda_top[d]):
        #if topic*100 > 10:
          print("Topic ",i,": ",topic*100,"%")

Document:  0
Topic  0 :  0.7685494551425659 %
Topic  1 :  0.7685530705603292 %
Topic  2 :  93.08304567096454 %
Topic  3 :  0.7685594905854223 %
Topic  4 :  0.7685462187689596 %
Topic  5 :  0.7685474615899363 %
Topic  6 :  0.7685487364627648 %
Topic  7 :  0.768549799432425 %
Topic  8 :  0.7685466305142927 %
Topic  9 :  0.7685534659787783 %


As we can see Topic 3 is dominantly present in document 0.

In [32]:
print(lda_model.components_[0])
print(lda_model.components_.shape)  # (no_of_topics*no_of_words)

[0.8703105  0.84144624 0.81044013 0.8104415  0.96441651 0.89323068
 0.79177836 0.87416348 0.84911142 0.68366097 0.74981356 0.85509045
 0.95423384 0.81110996 0.78590001 0.83834538 0.78150499 0.80584969
 0.82820951 0.74641686 0.89794658 0.75229537 0.72616902 0.84539531
 0.89075872 0.84329978 0.8199509  0.80510052 0.79247382 0.91820822
 0.87922274 0.7984369  0.77557227 0.90039325 0.91593652 0.90730014
 0.76305258 0.80445544 0.7910155  0.81432288 0.92040441 0.94449339
 0.85902586 0.77804186 0.82639954 0.96310608 0.83639389 0.80526738
 0.81159825 0.87493698 0.76541199 0.80737763 0.90593017 0.85629366
 0.83721324 0.93384628 0.77362452 0.80298904 0.85358833 0.85068791
 0.71972336 0.81894264 0.81627917 0.86255839 0.85039829 0.82327508
 0.68327031 0.82715401 0.83420549 1.04622012 0.81378549 0.85403784
 0.92568754 0.89190727 0.9486296  0.72072026 0.76779217 0.78418591
 0.70979863 0.83488503 0.7569167  0.96178432 0.76735399 0.80343398
 0.8971727  0.73341071 0.7056587  0.84438987 0.85059096 0.8944

Most important words for a topic. (say 5 this time.)

In [33]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:5]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
הוחזר מזעקי עודד רבים שעליה 

Topic 1: 
משקי לשאר מבין תקרית מאורע 

Topic 2: 
האחרות עלולה קפאותקוראים נצחונות שפירסם 

Topic 3: 
שהתאמנה שהביא וכבר מעולי להעריך 

Topic 4: 
מפסידים לשטח להרגיע אבטלה למכבי 

Topic 5: 
שמריהן והמיון שהבינו קיבל לקליטת 

Topic 6: 
מחוסרי סיכויים מוצלחת עירונית לשישה 

Topic 7: 
הויגמן לצוות לסכם קיימה לשגרתי 

Topic 8: 
שהגיע לעומת שסימלה מחויבות ובגיל 

Topic 9: 
שונא הוותיק בשני שהשתלבה למועדון 



In [34]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(vect_text)
type(lda_output)



numpy.ndarray

In [35]:
print(type(lda_output))
column_sums = lda_output.sum(axis=0)
print(column_sums)
column_sums.sum(axis=0)
column_sums/column_sums.sum(axis=0)

<class 'numpy.ndarray'>
[0.96700294 0.09414637 2.80581133 1.88035426 1.00979735 0.09414856
 0.09414658 1.0160198  1.01800864 1.02056416]


array([0.09670029, 0.00941464, 0.28058113, 0.18803543, 0.10097974,
       0.00941486, 0.00941466, 0.10160198, 0.10180086, 0.10205642])

In [36]:
lsa_output = lsa_model.transform(vect_text)
type(lsa_output)
print(type(lsa_output))
column_sums = lsa_output.sum(axis=0)
print(type(column_sums))
print(column_sums)
column_sums.sum(axis=0)
column_sums/column_sums.sum(axis=0)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ 4.23986239 -0.20143417  0.7402205   0.27622543  0.08913518 -0.24794485
  0.05241789  0.19585645 -0.18730454  0.06860304]


array([ 0.84364671, -0.04008132,  0.14728888,  0.05496326,  0.01773609,
       -0.049336  ,  0.0104301 ,  0.03897146, -0.03726981,  0.01365062])

![title](img\Afeka2.png)